In [ ]:
import WISScrapingTools as wis
import pandas as pd
import panel as pn
from io import StringIO, BytesIO

pn.extension()

In [ ]:
def app():
    df = wis.getAllSuikeis()
    names = [str(i) + '_' + n for i, n in zip(df.id.values[1:], df.name.values[1:])]
    dfn = pd.DataFrame({'name':names, 'id':df.id.values[1:]})
    
    select = pn.widgets.Select(name='name')
    select.options = dfn.name.values.tolist()
    
    button1 = pn.widgets.Button(name='download', button_type='primary') 
    obj = pn.Column('# WIS location download', select, button1, '')
    
    def download(event):
        # 作成した段階で実行されることに注意する。
        obj[-1] = 'Wait a minute.'
        
        i = dfn[dfn['name'] == select.value].id.squeeze()
        dff = wis.getAllStationsInSuikei(i, verbose=True, GeoDataFrame=True)
    
        sio = BytesIO()
        d = dff.to_file(sio, driver='GeoJSON', encoding='UTF-8')
        sio.seek(0)
        
        file_output = pn.widgets.FileDownload(sio, embed=True, filename=str(i)+'.geojson') 
        obj[-1] = file_output
    
    button1.on_click(download)

    return obj
    

In [ ]:
app().servable(title='app')